In [99]:
import numpy as np
from PIL import Image

In [100]:
def invert_channel(img):
    bgr = img.copy()
    # to convert from RGB to BGR
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            bgr[i][j][2] = img[i][j][0]
            bgr[i][j][1] = img[i][j][1]
            bgr[i][j][0] = img[i][j][2]
    return bgr

In [101]:
def color2gray(colour_img):
    r, g, b = colour_img[:,:,0], colour_img[:,:,1], colour_img[:,:,2]
    gray_img = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return gray_img

In [102]:
def threshold(img, thresh, value, else_value):   
    new = np.empty((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    
    for x in range(img.shape[0]):
        for y in range(img.shape[1]):
            if img[x,y] > thresh:
                new[x,y, :] = value
            else:
                new[x,y, :] = else_value
    return new

In [106]:
img1 = np.array(Image.open('roi.jpg'))
# extracting the ball region
img2 = img1[880:1040, 1030:1190,:]
# converting the image from rgb to bgr
img1 = invert_channel(img1)
img2 = invert_channel(img2)
# Region in img1 where i want to put the ball image
r, w, c = img2.shape
roi_start_x = 855
roi_start_y = 220
roi = img1[roi_start_x:roi_start_x+r, roi_start_y:roi_start_y+w, :]

#gray = cv.cvtColor(img2,cv.COLOR_BGR2GRAY)
gray = color2gray(img2)
mask_body = threshold(gray, 120, 255, 0)
mask_design = threshold(gray, 100, 0, 255)
mask = np.bitwise_or(mask_design, mask_body)

mask_inv = np.bitwise_not(mask)

# Now black-out the area of logo in ROI
img1_bg = np.bitwise_and(roi, mask_inv)

# Take only region of logo from logo image.
img2_fg = np.bitwise_and(img2, mask)

# Put logo in ROI and modify the main image
merged = np.bitwise_or(img1_bg, img2_fg)
img1[roi_start_x:roi_start_x+r, roi_start_y:roi_start_y+w, :] = merged

# converting the image from bgr to rgb
final = invert_channel(img1)
final = Image.fromarray(np.uint8(final))
final.save("ROI-output.png")
final.show()